In [0]:
import numpy as np
import cv2 as cv
import cv2
from matplotlib import pyplot as plt
from google.colab.patches import cv2_imshow
from google.colab import files
from IPython.display import Image

# command to remove files under the uploaded folder for uploading the same files would result in confusion sometimes
#!rm *.jpg
#!rm *.png
#!ls

uploaded = files.upload()
# queryImage
img1_rgb = cv.imread('box.jpg') # 0: loads image in grayscale mode         
uploaded = files.upload()
# trainImage
img2_rgb = cv.imread('box_btm_right.jpg') # 0: loads image in grayscale mode

# for panorama example
img1_rgb_black = np.zeros_like(img1_rgb)
img2_rgb = np.hstack((img1_rgb_black,img2_rgb))


img1 = cv2.cvtColor(img1_rgb, cv2.COLOR_BGR2GRAY)
img2 = cv2.cvtColor(img2_rgb, cv2.COLOR_BGR2GRAY)

cv2_imshow(img1_rgb)
#cv2_imshow(img1_rgb_black)
cv2_imshow(img2_rgb)






In [0]:

###
def drawMatches_self(img1, kp1, img2, kp2, matches):

    # Create a new output image that concatenates the two images together    
    rows1, cols1 = img1.shape    
    rows2, cols2 = img2.shape
    
    out = np.zeros((max([rows1,rows2]),cols1+cols2,3), dtype='uint8')

    # Place the first image in the form of RGB to the left
    out[:rows1,:cols1] = np.dstack([img1, img1, img1])

    # Place the next image in the form of RGB to the right of the 1st image
    out[:rows2,cols1:] = np.dstack([img2, img2, img2])
    
    
    # For each correspondence point, draw circles and connect a line between them
    for mat in matches:

        # Get the matching keypoints for each of the images
        img1_idx = mat.queryIdx
        img2_idx = mat.trainIdx
                
        # x - columns
        # y - rows
        (x1,y1) = kp1[img1_idx].pt
        (x2,y2) = kp2[img2_idx].pt        

        # random color for each pair of correspondence
        a = np.random.randint(0,256)
        b = np.random.randint(0,256)
        c = np.random.randint(0,256)

        # draw circles
        cv2.circle(out, (int(np.round(x1)),int(np.round(y1))), 2, (a, b, c), 5)      # random color with radius = 1
        cv2.circle(out, (int(np.round(x2)+cols1),int(np.round(y2))), 2, (a, b, c), 5)

        # Draw a line in-between each pair of correspondence                
        cv2.line(out, (int(np.round(x1)),int(np.round(y1))), (int(np.round(x2)+cols1),int(np.round(y2))), (a, b, c), 1) # random color with thickness = 1

    # Also return the image if you'd like a copy
    return out

###



# Initiate SIFT detector
#sift = cv.xfeatures2d.SIFT_create()

# find the keypoints and descriptors with SIFT
#kp1, des1 = sift.detectAndCompute(img1,None)
#kp2, des2 = sift.detectAndCompute(img2,None)

# Initiate ORB detector
orb = cv.ORB_create()
# find the keypoints and descriptors with ORB
kp1, des1 = orb.detectAndCompute(img1,None)
kp2, des2 = orb.detectAndCompute(img2,None)

# create BFMatcher object
bf = cv.BFMatcher(cv.NORM_HAMMING, crossCheck=True)
# Match descriptors.
matches = bf.match(des1,des2)

# Sort them in the order of their distance.
matches = sorted(matches, key = lambda x:x.distance)

 


fig=plt.figure(figsize=(15, 15))

# Draw matches. # official implementation
img3 = cv.drawMatches(img1,kp1,img2,kp2,matches[:50] ,None, flags=2, matchColor = (0,0,255))
# Draw matches. self implementation for knowing the detail
#img3 = drawMatches_self(img1,kp1,img2,kp2,matches[:10])

plt.imshow(img3)
plt.show()

In [0]:
import random
import math

def homo_estimation(feature_correspondence_in_query_image, feature_correspondence_in_train_image, desired_num_correspondence_homo):
  # A in AX=0
  MatrixA = np.zeros((desired_num_correspondence_homo*2,9))
  
  # The Homography transformation matrix
  HomoMat = np.zeros((3,3))
  
  rows, _ = feature_correspondence_in_query_image.shape
  
  rand_num = random.sample(range(rows), desired_num_correspondence_homo)
  #print(rand_num)
  
  
  feature_correspondence_in_query_image_randomly_picked = np.zeros((desired_num_correspondence_homo,2))
  feature_correspondence_in_train_image_randomly_picked = np.zeros((desired_num_correspondence_homo,2))      
  
  for i in range(desired_num_correspondence_homo):
    feature_correspondence_in_query_image_randomly_picked.itemset((i,0),feature_correspondence_in_query_image.item(rand_num[i],0))
    feature_correspondence_in_query_image_randomly_picked.itemset((i,1),feature_correspondence_in_query_image.item(rand_num[i],1))
    
    feature_correspondence_in_train_image_randomly_picked.itemset((i,0),feature_correspondence_in_train_image.item(rand_num[i],0))
    feature_correspondence_in_train_image_randomly_picked.itemset((i,1),feature_correspondence_in_train_image.item(rand_num[i],1))
    
  
            
  for i in range(desired_num_correspondence_homo):
    MatrixA.itemset((i*2,0),-feature_correspondence_in_query_image_randomly_picked.item(i,0))
    MatrixA.itemset((i*2,1),-feature_correspondence_in_query_image_randomly_picked.item(i,1))
    MatrixA.itemset((i*2,2),-1.0)
    MatrixA.itemset((i*2,3),0.0)
    MatrixA.itemset((i*2,4),0.0)
    MatrixA.itemset((i*2,5),0.0)
    MatrixA.itemset((i*2,6),feature_correspondence_in_train_image_randomly_picked.item(i,0)*feature_correspondence_in_query_image_randomly_picked.item(i,0))
    MatrixA.itemset((i*2,7),feature_correspondence_in_train_image_randomly_picked.item(i,0)*feature_correspondence_in_query_image_randomly_picked.item(i,1))
    MatrixA.itemset((i*2,8),feature_correspondence_in_train_image_randomly_picked.item(i,0))

    MatrixA.itemset((i*2+1,0),0.0)
    MatrixA.itemset((i*2+1,1),0.0)
    MatrixA.itemset((i*2+1,2),0.0)
    MatrixA.itemset((i*2+1,3),-feature_correspondence_in_query_image_randomly_picked.item(i,0))
    MatrixA.itemset((i*2+1,4),-feature_correspondence_in_query_image_randomly_picked.item(i,1))
    MatrixA.itemset((i*2+1,5),-1.0)
    MatrixA.itemset((i*2+1,6),feature_correspondence_in_train_image_randomly_picked.item(i,1)*feature_correspondence_in_query_image_randomly_picked.item(i,0))
    MatrixA.itemset((i*2+1,7),feature_correspondence_in_train_image_randomly_picked.item(i,1)*feature_correspondence_in_query_image_randomly_picked.item(i,1))
    MatrixA.itemset((i*2+1,8),feature_correspondence_in_train_image_randomly_picked.item(i,1))

  #print(MatrixA)    
  # performing SVD for estimating Homography matrix
  u, s, vh = np.linalg.svd(MatrixA, full_matrices=True)

  HomoMat.itemset((0,0),vh.item(8,0))
  HomoMat.itemset((0,1),vh.item(8,1))
  HomoMat.itemset((0,2),vh.item(8,2))

  HomoMat.itemset((1,0),vh.item(8,3))
  HomoMat.itemset((1,1),vh.item(8,4))
  HomoMat.itemset((1,2),vh.item(8,5))

  HomoMat.itemset((2,0),vh.item(8,6))
  HomoMat.itemset((2,1),vh.item(8,7))
  HomoMat.itemset((2,2),vh.item(8,8))
  
  #print(HomoMat)
                    
  return HomoMat     
                    
def transfer_error_estimation(HomoMat, feature_correspondence_in_query_image, feature_correspondence_in_train_image):
    input_vector = np.array([[0],[0],[0]])
    output_vector = np.array([[0],[0],[0]])
    #print(type(output_vector))
                    
    feature_num, _ = feature_correspondence_in_query_image.shape                    
    square_error = 0
    
    for i in range(feature_num):
        input_vector.itemset((0,0), feature_correspondence_in_query_image.item(i,0))
        input_vector.itemset((1,0), feature_correspondence_in_query_image.item(i,1))
        input_vector.itemset((2,0), 1.0)                    
  
        output_vector = np.matmul(HomoMat,input_vector)            
                    
        output_vector.itemset((0,0), output_vector.item(0,0)/output_vector.item(2,0))
        output_vector.itemset((1,0), output_vector.item(1,0)/output_vector.item(2,0))            
        
        #print(input_vector)
        #print(output_vector)
        
                    
        square_error = square_error + math.sqrt( (output_vector.item(0,0) - feature_correspondence_in_train_image.item(i,0) )**2 + ( output_vector.item(1,0) - feature_correspondence_in_train_image.item(i,1) )**2 )         
                    
    return square_error        

def RANSAC_fun(kp1,kp2,matches, num_correspondence, desired_num_correspondence_homo, iteration_num):
    
    
    
    num_matches = len(matches)
    feature_correspondence_in_query_image = np.zeros((num_matches,2))
    feature_correspondence_in_train_image = np.zeros((num_matches,2))
    
    ## for debug only
    #feature_correspondence_in_query_image = np.zeros((4,2))
    #feature_correspondence_in_train_image = np.zeros((4,2))
    ##
    running_index = 0
    
    
    
    for mat in matches:
        img1_idx = mat.queryIdx
        img2_idx = mat.trainIdx
                
        # x - columns
        # y - rows
        (x1,y1) = kp1[img1_idx].pt
        (x2,y2) = kp2[img2_idx].pt
        #print('kp1[img1_idx].pt={}'.format(kp1[img1_idx].pt))
        #print('kp2[img2_idx].pt={}'.format(kp2[img2_idx].pt))
        feature_correspondence_in_query_image.itemset((running_index,0),x1)
        feature_correspondence_in_query_image.itemset((running_index,1),y1)
      
        feature_correspondence_in_train_image.itemset((running_index,0),x2)
        feature_correspondence_in_train_image.itemset((running_index,1),y2)
      
        running_index = running_index + 1
    
    
    '''
    ## for debug only
    feature_correspondence_in_query_image.itemset((0,0),0)
    feature_correspondence_in_query_image.itemset((0,1),0)
    
    feature_correspondence_in_query_image.itemset((1,0),324)
    feature_correspondence_in_query_image.itemset((1,1),0)
    
    feature_correspondence_in_query_image.itemset((2,0),0)
    feature_correspondence_in_query_image.itemset((2,1),223)
    
    feature_correspondence_in_query_image.itemset((3,0),324)
    feature_correspondence_in_query_image.itemset((3,1),223)
    
    
    feature_correspondence_in_train_image.itemset((0,0),118)
    feature_correspondence_in_train_image.itemset((0,1),164)
    
    feature_correspondence_in_train_image.itemset((1,0),277)
    feature_correspondence_in_train_image.itemset((1,1),180)
    
    feature_correspondence_in_train_image.itemset((2,0),92)
    feature_correspondence_in_train_image.itemset((2,1),262)
    
    feature_correspondence_in_train_image.itemset((3,0),265)
    feature_correspondence_in_train_image.itemset((3,1),294)
    ##
    '''
    
    iteration_index  = 0  
    transfer_error_best = 100000
    
    while(iteration_index<iteration_num):
    
      HomoMat = homo_estimation(feature_correspondence_in_query_image, feature_correspondence_in_train_image, desired_num_correspondence_homo)
    
      transfer_error = transfer_error_estimation(HomoMat, feature_correspondence_in_query_image, feature_correspondence_in_train_image)
      #print(transfer_error)
      if transfer_error < transfer_error_best:
        transfer_error_best =  transfer_error
        best_HomoMat = np.copy(HomoMat)
    
      iteration_index = iteration_index + 1
    
    #print(transfer_error_best)
    return best_HomoMat
                    
    #print(rand_num)
#####################################    


######################################
  
def CoordinateTransformation(SimTranMax,y,x):
    input_vector = np.array([[0],[0],[0]])
    output_vector = np.array([[0],[0],[0]])
    #print(type(output_vector))
    
    input_vector.itemset((0,0), x)
    input_vector.itemset((1,0), y)
    input_vector.itemset((2,0), 1.0)
    
    output_vector = np.matmul(SimTranMax,input_vector)
    
    output_vector.itemset((0,0), output_vector.item(0,0)/output_vector.item(2,0))
    output_vector.itemset((1,0), output_vector.item(1,0)/output_vector.item(2,0))
    return output_vector
  
  
  
def ImageTransformation(TranMax,src,result_img):
    
    h, w ,_ = src.shape
    h1, w1 ,_ = result_img.shape
    # output image
    #result_img = np.zeros_like(src)
    
    for i in range(h):
        for j in range(w):
        
            pixel_R = src.item(i,j,0)
            pixel_G = src.item(i,j,1)
            pixel_B = src.item(i,j,2)           
            
            output_vector = CoordinateTransformation(TranMax,i,j)
            
            output_vector = output_vector.astype(int)
            
            output_vector_x = output_vector.item(0,0)
            output_vector_y = output_vector.item(1,0)            
        
            if output_vector_x >= 0 and output_vector_x < w1 and output_vector_y >= 0 and output_vector_y < h1:
                result_img.itemset((output_vector_y,output_vector_x,0),pixel_R)
                result_img.itemset((output_vector_y,output_vector_x,1),pixel_G)
                result_img.itemset((output_vector_y,output_vector_x,2),pixel_B)
            
    return result_img


## 1. self-implemented

## eliminate poor pairs
GOOD_MATCH_PERCENT = 0.15
numGoodMatches = int(len(matches) * GOOD_MATCH_PERCENT)
matches_selected = matches[:numGoodMatches]
  
## the number of matches 
num_correspondence = len(matches_selected[:])

## setting the number of feature correspondence to be fed into homography estimation
desired_num_correspondence_homo = 8

## setting the iteration number
iteration_num = 1000

best_HomoMat = RANSAC_fun(kp1,kp2,matches_selected, num_correspondence, desired_num_correspondence_homo, iteration_num)

## make an RGB image from grayscale image
# Place the first image in the form of RGB to the left
#img1_rgb = np.dstack([img1, img1, img1])

# Place the next image in the form of RGB to the right of the 1st image
#img2_rgb = np.dstack([img2, img2, img2])

## draw polygon for clarity
#h,w = img1.shape[:2]
#pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2) # add a new dimension by -1
#dst = cv2.perspectiveTransform(pts,best_HomoMat)
#img2_rgb = cv2.polylines(img2_rgb, [np.int32(dst)], True, (0,0,255), 5, cv2.LINE_AA)
##

result_img = ImageTransformation(best_HomoMat,img1_rgb,img2_rgb)  

cv2_imshow(result_img)


## concise implementation by calling OpenCV functions
'''
src_pts  = np.float32([kp1[m.queryIdx].pt for m in matches_selected]).reshape(-1,1,2)
dst_pts  = np.float32([kp2[m.trainIdx].pt for m in matches_selected]).reshape(-1,1,2)

## find homography matrix and do perspective transform
M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
h,w = img1.shape[:2]
pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
dst = cv2.perspectiveTransform(pts,M)

## draw found regions
img2 = cv2.polylines(img2, [np.int32(dst)], True, (0,0,255), 1, cv2.LINE_AA)
cv2_imshow(img2)

## draw match lines
res = cv2.drawMatches(img1, kp1, img2, kp2, matches_selected[:20],None,flags=2)

cv2_imshow(res)
'''